# Imports

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [2]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 13                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data_new"
DEPRESJON_PREFIX = "manual_depresjon"
HYPERAKTIV_PREFIX = "manual_hyperaktiv"
PSYKOSE_PREFIX = "manual_psykose"

# Manual feature extraction

## Helper functions

In [3]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/13),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/13),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [4]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [5]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [6]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 13hs
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "day", corresponding to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("day", full_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [7]:
dataset = Dataset(dirpath=os.path.join("data_day", "depresjon"))
condition = dataset.condition
control = dataset.control

In [8]:
condition[0]

,timestamp,date,activity
0,2003-05-08 08:00:00,2003-05-08,3
1,2003-05-08 08:01:00,2003-05-08,0
2,2003-05-08 08:02:00,2003-05-08,0
3,2003-05-08 08:03:00,2003-05-08,0
4,2003-05-08 08:04:00,2003-05-08,0
...,...,...,...
775,2003-05-08 20:55:00,2003-05-08,52
776,2003-05-08 20:56:00,2003-05-08,20
777,2003-05-08 20:57:00,2003-05-08,5
778,2003-05-08 20:58:00,2003-05-08,0


In [9]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [10]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [11]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [12]:
datasets["day"].dropna(inplace=True)
datasets["day"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,75.916664,395.649994,265.802551,279.883331,12065.484375,-1.171582,-0.431376,0.397038,187.633347,271.260620,...,2.998658e+05,3.341706e+05,4.923075e+10,-1.275133,0.156652,0.739931,3.461079e+05,2.998658e+05,2.332665,0.502459
1,7.733333,432.483337,197.839752,192.399994,15148.465820,-0.583989,0.079738,0.597709,115.433334,193.790909,...,3.570634e+05,1.394110e+05,1.262747e+11,-0.791910,0.868935,0.995206,4.617640e+05,3.570634e+05,2.123986,0.543610
2,60.083332,447.583344,214.744858,206.699997,12191.085938,-0.223079,0.590566,0.493989,74.533340,207.637894,...,1.052825e+05,1.185609e+05,3.691578e+09,-1.002532,0.076838,0.577098,8.095201e+04,1.052825e+05,2.537144,0.777841
3,57.200001,299.966675,162.461533,152.283340,5171.954590,-0.538682,0.400807,0.425300,72.949997,159.530289,...,1.598933e+05,1.536467e+05,8.813271e+09,-1.191926,0.013451,0.587135,1.376897e+05,1.598933e+05,2.515386,0.724468
4,16.900000,1137.300049,562.648743,368.250000,138058.640625,-1.400442,0.168262,0.634473,574.883362,560.021240,...,2.241901e+06,1.525539e+06,1.605035e+12,-1.177838,0.646454,0.565101,1.875858e+06,2.241901e+06,2.587361,0.854907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014,0.000000,0.083333,0.006410,0.000000,0.000534,8.083334,3.175426,3.464102,0.000000,0.000000,...,2.323129e-02,2.868030e-02,8.784988e-05,0.528203,-1.451612,0.403457,6.180678e-03,2.323129e-02,2.638663,0.807204
1015,0.000000,124.833336,23.084616,0.000000,1527.869263,1.455530,1.588439,1.626820,36.966667,15.933333,...,1.597886e+04,4.985574e+03,4.166649e+08,0.620137,1.376749,1.277461,2.029612e+04,1.597886e+04,1.755936,0.316412
1018,0.000000,0.583333,0.044872,0.000000,0.026175,8.083336,3.175428,3.464101,0.000000,0.000000,...,1.482524e+00,1.860298e+00,3.682968e-01,0.002368,-1.275860,0.409352,5.058052e-01,1.482523e+00,2.641863,0.825876
1019,0.000000,0.816667,0.065385,0.000000,0.051040,8.047117,3.166283,3.319695,0.000000,0.003030,...,1.297386e+00,1.519873e+00,2.557160e-01,1.563398,-1.788572,0.389771,2.115891e-01,1.297386e+00,2.637340,0.780309


## HYPERAKTIV

In [13]:
dataset = Dataset(dirpath=os.path.join("data_day", "hyperaktiv"))
condition = dataset.condition
control = dataset.control

In [14]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [15]:
for part, df in datasets.items():
    filename = f"{HYPERAKTIV_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [16]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"hyperaktiv_y.csv")
y.to_csv(filepath, header=False, index=False)

In [17]:
datasets["day"].dropna(inplace=True)
datasets["day"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,5.450000,147.433334,62.847435,43.950001,2284.001221,-0.914054,0.728419,0.730600,72.199997,60.375755,...,38759.406250,21145.060547,2.928775e+09,1.929085,1.929875,1.396259,14845.820312,38759.406250,1.827267,0.496738
1,0.000000,9.133333,2.510257,2.450000,7.301598,0.679430,1.063763,1.034212,3.283333,2.136364,...,78.993980,91.679405,1.346396e+03,-1.501138,-0.275953,0.464507,61.959623,78.993980,2.633243,0.863299
2,0.000000,82.599998,15.712819,3.916667,611.214355,2.285171,1.848395,1.511686,12.950000,11.060606,...,28950.376953,25248.203125,2.239280e+08,-0.721994,0.360955,0.516892,18206.178711,28950.378906,2.605718,0.845312
3,2.533333,403.383331,134.980759,91.349998,16141.065430,-0.447485,0.889049,0.904301,162.066673,122.621216,...,150422.703125,97665.960938,1.761064e+10,-0.007812,1.150318,0.882214,127027.554688,150422.703125,2.306045,0.677691
4,7.500000,62.933334,32.089745,30.549999,320.231171,-0.919846,0.394668,0.535778,21.216667,31.521210,...,5731.346191,2009.573608,3.918533e+07,0.205687,1.285825,1.092207,6139.816406,5731.346191,2.076416,0.573473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,22.333334,550.450012,196.321777,135.466660,26017.556641,0.155375,1.174797,0.789376,164.333351,179.945450,...,174145.281250,81364.820312,1.823810e+10,-0.825156,0.761091,0.775493,190299.281250,174145.281250,2.393930,0.729809
510,44.416668,393.549988,152.557693,100.116669,12873.285156,-0.312746,0.961288,0.714545,141.866661,140.480301,...,91368.007812,53296.035156,6.254545e+09,0.120749,1.158702,0.865573,84590.821289,91368.000000,2.324360,0.691359
511,70.900002,528.700012,210.732040,194.216660,19874.033203,-0.017442,0.967725,0.642734,195.466667,194.537857,...,266050.906250,208450.796875,1.799971e+10,0.540082,1.227805,0.504276,105362.554688,266050.937500,2.642954,0.894867
512,52.366665,272.850006,116.820511,100.366669,4457.985840,0.607930,1.324658,0.549122,55.699997,108.495453,...,118946.867188,55876.003906,1.687564e+10,-0.832786,0.962896,1.092137,135598.316406,118946.867188,2.000861,0.389556


## Psykose

In [18]:
dataset = Dataset(dirpath=os.path.join("data_day", "psykose"))
condition = dataset.condition
control = dataset.control

In [19]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [20]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [21]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)

In [22]:
datasets["day"].dropna(inplace=True)
datasets["day"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,182.016663,777.866638,393.317902,311.666656,38946.679688,-0.440009,1.067308,0.482070,92.000000,377.568176,...,853306.875000,513316.750000,6.115879e+11,-0.451251,0.879144,0.916483,8.992948e+05,853306.875000,2.197079,0.511063
1,137.066666,796.299988,337.185852,257.633331,40202.367188,0.435714,1.288159,0.571315,94.299988,313.640900,...,874098.875000,794452.312500,5.118297e+11,-1.304732,0.527845,0.818469,1.123060e+06,874098.812500,2.303899,0.619114
2,7.083333,349.533325,144.451279,161.866669,11407.171875,-0.803866,0.181210,0.710373,176.199999,138.295456,...,124669.250000,97995.875000,8.850755e+09,0.093137,1.048304,0.754624,7.435558e+04,124669.250000,2.414316,0.698076
3,13.400000,445.533325,210.153839,200.116669,12142.401367,0.076977,0.376242,0.503772,127.050003,206.642395,...,93026.328125,74884.773438,3.760575e+09,0.043230,1.112833,0.659206,5.503534e+04,93026.328125,2.525348,0.818834
4,43.750000,263.883331,167.435898,180.949997,4139.375488,-0.776706,-0.352753,0.369180,89.400002,169.912125,...,96791.414062,99770.054688,2.228271e+09,-0.961162,-0.027109,0.487694,5.841487e+04,96791.390625,2.614350,0.837097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,0.000000,0.083333,0.006410,0.000000,0.000534,8.083334,3.175426,3.464102,0.000000,0.000000,...,0.023231,0.028680,8.784988e-05,0.528203,-1.451612,0.403457,6.180678e-03,0.023231,2.638663,0.807204
1002,0.000000,124.833336,23.084616,0.000000,1527.869263,1.455530,1.588439,1.626820,36.966667,15.933333,...,15978.862305,4985.574219,4.166649e+08,0.620137,1.376749,1.277461,2.029612e+04,15978.864258,1.755936,0.316412
1005,0.000000,0.583333,0.044872,0.000000,0.026175,8.083336,3.175428,3.464101,0.000000,0.000000,...,1.482524,1.860298,3.682968e-01,0.002368,-1.275860,0.409352,5.058052e-01,1.482523,2.641863,0.825876
1006,0.000000,0.816667,0.065385,0.000000,0.051040,8.047117,3.166283,3.319695,0.000000,0.003030,...,1.297386,1.519873,2.557160e-01,1.563398,-1.788572,0.389771,2.115891e-01,1.297386,2.637340,0.780309


# Classification

## Classifiers, parameters, constants

In [23]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    ),
    "LGBM": LGBMClassifier(
        n_estimators=200,
        verbosity=-1,
        random_state=0
    ),
    "XGB": XGBClassifier(
        n_estimators=200,
        random_state=0 
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": ["balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "C": [1, 10],
        "gamma": ["scale"],
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    },
    "LGBM": {
        "num_leaves": [31, 50],
        "min_child_samples": [10, 20],
        "class_weight": [None, "balanced"]
    },
    "XGB": {
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 6],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "scale_pos_weight": [1, 10]
    }
}

## Classification Depresjon

In [24]:
dataset = DEPRESJON_PREFIX
y_filename = "depresjon_y.csv"

In [25]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)

In [26]:
print("Wyniki dla depresjon")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]

    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            #print(train_idx, test_idx)
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            y_pred  = clf.predict(X_test)
            metrics = calculate_metrics(y_test, y_pred)
            #print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla depresjon
PART: day
  LR
    accuracy: 0.4242 +- 0.1027
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.4345 +- 0.2173
    precision: 0.4983 +- 0.2509
    recall: 0.8000 +- 0.4000
    specificity: 0.2000 +- 0.4000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  SVM
    accuracy: 0.6382 +- 0.0288
    balanced_accuracy: 0.5266 +- 0.0582
    f1: 0.1087 +- 0.2043
    precision: 0.7665 +- 0.3170
    recall: 0.0902 +- 0.1733
    specificity: 0.9630 +- 0.0586
    ROC_AUC: 0.5266 +- 0.0582
    MCC: 0.0515 +- 0.1339

  RF
    accuracy: 0.5570 +- 0.0574
    balanced_accuracy: 0.5226 +- 0.0495
    f1: 0.3962 +- 0.0484
    precision: 0.4097 +- 0.0692
    recall: 0.3881 +- 0.0462
    specificity: 0.6571 +- 0.0864
    ROC_AUC: 0.5226 +- 0.0495
    MCC: 0.0484 +- 0.1016

  LGBM
    accuracy: 0.5970 +- 0.0357
    balanced_accuracy: 0.5474 +- 0.0437
    f1: 0.3916 +- 0.0775
    precision: 0.4430 +- 0.0636
    recall: 0.3537 +- 0.0853
    specificity: 0.7412 +- 0.0342
    ROC_AUC

# Classification HYPERAKTIV

In [27]:
dataset = HYPERAKTIV_PREFIX
y_filename = "hyperaktiv_y.csv"

In [28]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)

In [29]:
print("Wyniki dla depresjon")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]

    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            #print(train_idx, test_idx)
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.03)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            y_pred  = clf.predict(X_test)
            metrics = calculate_metrics(y_test, y_pred)
            #print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla depresjon
PART: day
  LR
    accuracy: 0.5264 +- 0.0471
    balanced_accuracy: 0.5279 +- 0.0484
    f1: 0.4716 +- 0.0333
    precision: 0.5592 +- 0.0812
    recall: 0.4192 +- 0.0613
    specificity: 0.6366 +- 0.1343
    ROC_AUC: 0.5279 +- 0.0484
    MCC: 0.0631 +- 0.1056

  SVM
    accuracy: 0.5595 +- 0.0371
    balanced_accuracy: 0.5592 +- 0.0386
    f1: 0.5769 +- 0.0241
    precision: 0.5716 +- 0.0566
    recall: 0.5962 +- 0.0798
    specificity: 0.5223 +- 0.1400
    ROC_AUC: 0.5592 +- 0.0386
    MCC: 0.1218 +- 0.0796

  RF
    accuracy: 0.5284 +- 0.0661
    balanced_accuracy: 0.5281 +- 0.0669
    f1: 0.5488 +- 0.0457
    precision: 0.5403 +- 0.0756
    recall: 0.5615 +- 0.0283
    specificity: 0.4947 +- 0.1233
    ROC_AUC: 0.5281 +- 0.0669
    MCC: 0.0564 +- 0.1350

  LGBM
    accuracy: 0.4954 +- 0.0638
    balanced_accuracy: 0.4950 +- 0.0638
    f1: 0.5142 +- 0.0609
    precision: 0.5030 +- 0.0621
    recall: 0.5269 +- 0.0627
    specificity: 0.4631 +- 0.0802
    ROC_AUC

# Classification Psykose

In [30]:
dataset = PSYKOSE_PREFIX
y_filename = "psykose_y.csv"

In [31]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)

In [32]:
print("Wyniki dla psykose")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]

    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            #print(train_idx, test_idx)
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            y_pred  = clf.predict(X_test)
            metrics = calculate_metrics(y_test, y_pred)
            #print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla psykose
PART: day
  LR
    accuracy: 0.3561 +- 0.0014
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.5251 +- 0.0015
    precision: 0.3561 +- 0.0014
    recall: 1.0000 +- 0.0000
    specificity: 0.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  SVM
    accuracy: 0.6429 +- 0.0008
    balanced_accuracy: 0.4992 +- 0.0017
    f1: 0.0000 +- 0.0000
    precision: 0.8000 +- 0.4000
    recall: 0.0000 +- 0.0000
    specificity: 0.9983 +- 0.0034
    ROC_AUC: 0.4992 +- 0.0017
    MCC: -0.0109 +- 0.0219

  RF
    accuracy: 0.5497 +- 0.0303
    balanced_accuracy: 0.4915 +- 0.0137
    f1: 0.3102 +- 0.0346
    precision: 0.3470 +- 0.0215
    recall: 0.2889 +- 0.0572
    specificity: 0.6941 +- 0.0769
    ROC_AUC: 0.4915 +- 0.0137
    MCC: -0.0163 +- 0.0285

  LGBM
    accuracy: 0.5800 +- 0.0257
    balanced_accuracy: 0.5183 +- 0.0289
    f1: 0.3397 +- 0.0436
    precision: 0.3858 +- 0.0469
    recall: 0.3038 +- 0.0418
    specificity: 0.7328 +- 0.0234
    ROC_AUC